In [37]:
import tensorflow.compat.v1 as tf

tf.disable_eager_execution()

In [38]:
%xmode plain

Exception reporting mode: Plain


## Variables

In [17]:
# here W is a Variable
W = tf.Variable([2.5, 4.0], tf.float32, name='var_W')
x = tf.placeholder(tf.float32, name='x')
# b is also a variable with initial value 5 and 10
b = tf.Variable([5.0, 10.0], tf.float32, name='var_b')
y = W * x + b

# initialize all variables defined
init = tf.global_variables_initializer()
# use 'with' statement to instantiate and assign a session
with tf.Session() as sess:
    sess.run(init)
    # this computation is required to initialize the variable
    print("Final result: Wx + b = ", sess.run(y, feed_dict={x: [10,100]}))

Final result: Wx + b =  [ 30. 410.]


In [5]:
W

<tf.Variable 'var_W_1:0' shape=(2,) dtype=float32>

## Multiple graphs

## `as_default()`
Returns a context manager that makes this Graph the default graph.

This method should be used if you want to create multiple graphs in the same process. For convenience, a global default graph is provided, and all ops will be added to this graph if you do not create a new graph explicitly.

Use this method with the with keyword to specify that ops created within the scope of a block should be added to this graph. In this case, once the scope of the with is exited, the previous default graph is set again as default. There is a stack, so it's ok to have multiple nested levels of as_default calls.

The default graph is a property of the current thread. If you create a new thread, and wish to use the default graph in that thread, you must explicitly add a with `g.as_default()`: in that thread's function.

The following code examples are equivalent:


### 1. Using Graph.as_default():
```python
g = tf.Graph()
with g.as_default():
  c = tf.constant(5.0)
  assert c.graph is g
```

### 2. Constructing and making default:
```python
with tf.Graph().as_default() as g:
  c = tf.constant(5.0)
  assert c.graph is g
```
If eager execution is enabled ops created under this context manager will be added to the graph instead of executed eagerly.

In [30]:
g1 = tf.Graph()
with g1.as_default():
    with tf.Session() as sess:
        A = tf.constant([5,7],tf.int32, name='A')
        x = tf.placehoalder(tf.int32, name='x')
        b = tf.constant([3,4],tf.int32, name='b')
        y = A * x + b
        print(sess.run(y, feed_dict={x: [10, 100]}))
assert y.graph is g1

[ 53 704]


In [57]:
A = tf.constant([5,7],tf.int32, name='A')
x = tf.placeholder(tf.int32, name='x')
b = tf.constant([3,4],tf.int32, name='b')
A, x, b

(<tf.Tensor 'A_8:0' shape=(2,) dtype=int32>,
 <tf.Tensor 'x_11:0' shape=<unknown> dtype=int32>,
 <tf.Tensor 'b_8:0' shape=(2,) dtype=int32>)

In [64]:
#default_graph
default_graph.get_tensor_by_name('A_8:0') # op_name: index

<tf.Tensor 'A_8:0' shape=(2,) dtype=int32>

In [48]:
g1 = tf.Graph()
with g1.as_default():
    sess = tf.Session()
#     A = tf.constant([5,7],tf.int32, name='A')
#     x = tf.placeholder(tf.int32, name='x')
#     b = tf.constant([3,4],tf.int32, name='b')
    y = A * x + b
    print(sess.run(y, feed_dict={x: [10, 100]}))

RuntimeError: The Session graph is empty.  Add operations to the graph before calling run().

In [70]:
g2 = tf.Graph()
with g2.as_default():
    with tf.Session() as sess:
        A = tf.constant([5,7],tf.int32, name='A')
        x = tf.placeholder(tf.int32, name='x')
        y = tf.pow(A, x, name='y')
        print( sess.run(y, feed_dict={x: [3, 5]}))
assert y.graph is g2

[  125 16807]


In [71]:
A, x, y

(<tf.Tensor 'A:0' shape=(2,) dtype=int32>,
 <tf.Tensor 'x:0' shape=<unknown> dtype=int32>,
 <tf.Tensor 'y:0' shape=<unknown> dtype=int32>)

In [72]:
default_graph = tf.get_default_graph()
assert default_graph is g2

AssertionError: 

In [67]:
with tf.Session() as sess:
    A = default_graph.get_tensor_by_name('A:0')
    x = default_graph.get_tensor_by_name('x:0')
#     A = tf.constant([5,7], tf.int32, name='A')
#     x = tf.placeholder(tf.int32, name='x')
    y = A + x
    print(sess.run(y, feed_dict={x: [3, 5]}))
assert y.graph is default_graph

TypeError: Input 'y' of 'AddV2' Op has type float32 that does not match type int32 of argument 'x'.